In [17]:
from datetime import datetime, timedelta
import time
import requests
import csv

In [ ]:
Creating variables that hold the coordinates of the city Bangalore, the API key provided by DarkSky API, the Base URL, that starting date fr 

In [18]:
LOCATION = "12.976,77.954"
API_KEY = 'a086db0044d848560a257227c648e59f'
BASE_URL = "https://api.darksky.net/forecast/{}/{},{}?{}"
EXCLUDE = "exclude=currently,minutely,hourly,alerts,flags"
TARGET_DATE = datetime(2018, 1, 1)

features = ["date", "temperatureMin", "temperatureMax", "humidity", "dewPoint", "windSpeed"]

In [ ]:
def extract_weather_data(url, api_key, location, exclude, target_date, days):
    record = []
    for _ in range(days):
        rec_day = []
        string_time = target_date.strftime('%Y/%m/%d')
        response = requests.get(url.format(api_key, location, int(time.mktime(datetime.strptime(string_time,'%Y/%m/%d').timetuple())), exclude))
        if response.status_code == 200:
            
            data = response.json()['daily']['data'][0]
            rec_day.append(target_date.strftime('%Y/%m/%d'))
            for item in features:
                if item != 'date':
                    rec_day.append(data[item])
                
        record.append(rec_day)
        time.sleep(6)
        target_date += timedelta(1)
    return record

In [ ]:
with open('WeatherDataSet.csv', 'a+', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(features)
    
    collected_data = extract_weather_data(BASE_URL, API_KEY, LOCATION, EXCLUDE, TARGET_DATE, 940)
    for day in collected_data:
        csv_writer.writerow(day)
        
csv_file.close()

In [ ]:
TARGET_DATE += timedelta(940)
with open('WeatherDataSet.csv', 'a+', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(features)
    
    collected_data = extract_weather_data(BASE_URL, API_KEY, LOCATION, EXCLUDE, TARGET_DATE, 886)
    for day in collected_data:
        csv_writer.writerow(day)
        
csv_file.close()

In [42]:
EXCLUDE = "exclude=currently,minutely,daily,alerts,flags"
TARGET_DATE = datetime(2018, 1, 1)

In [41]:
def get_mean_temp(url, api_key, location, exclude, target_date, days):
    meanTemp = []
    for _ in range(days):
        temp = 0
        string_time = target_date.strftime('%Y/%m/%d')
        response = requests.get(url.format(api_key, location, int(time.mktime(datetime.strptime(string_time,'%Y/%m/%d').timetuple())), exclude))
        if response.status_code == 200:
            for i in range(24):
                data = response.json()['hourly']['data'][i]
                temp += data["temperature"]
                data ={}
            mean = temp/24
            meanTemp.append(round(mean,4))
    return meanTemp

In [ ]:
collectedMeanTemp = get_mean_temp(BASE_URL, API_KEY, LOCATION, EXCLUDE, TARGET_DATE, 940)

In [13]:
with open('tempfile.csv', 'r') as csv_file:
    for line in csv_file.readlines():
        collectedMeanTemp = list(map(float,line.strip().split(',')))

In [ ]:
print(collectedMeanTemp)

In [22]:
TARGET_DATE += timedelta(940)
collectedMeanTemp2 = get_mean_temp(BASE_URL, API_KEY, LOCATION, EXCLUDE, TARGET_DATE, 886)

In [ ]:
print(collectedMeanTemp2)

In [38]:
meanTemp = collectedMeanTemp + collectedMeanTemp2

In [ ]:
print(meanTemp)

In [49]:
import pandas as pd
csv_data = pd.read_csv("WeatherDataSet.csv")
df = pd.DataFrame(csv_data)
df["temperatureMean"] = meanTemp

In [50]:
df

,date,temperatureMin,temperatureMax,humidity,dewPoint,windSpeed,temperatureMean
0,2018/01/01,58.78,83.07,0.67,57.63,3.36,70.5929
1,2018/01/02,60.46,83.13,0.69,59.33,4.55,70.5929
2,2018/01/03,64.04,80.68,0.70,60.35,4.48,70.5929
3,2018/01/04,60.63,80.30,0.71,58.65,5.91,70.5929
4,2018/01/05,56.63,79.61,0.65,54.36,4.81,70.5929
...,...,...,...,...,...,...,...
1821,2022/12/27,62.03,79.70,0.69,58.38,4.77,75.2133
1822,2022/12/28,62.02,79.69,0.69,58.27,4.77,75.2133
1823,2022/12/29,62.01,79.69,0.68,58.16,4.77,75.2133
1824,2022/12/30,62.01,79.69,0.68,58.05,4.77,75.2133


In [52]:
df.to_csv('WeatherData.csv', index=False)